In [ ]:
%pip install mediapipe

In [ ]:
!python3 --version
!python3 -m pip --version

In [ ]:
!pip install --upgrade pip

In [ ]:
import tensorflow as tf

In [1]:
import os 
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp

# KeyPoints using MP holistic


In [ ]:
mp_holistic = mp.solutions.holistic  # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [ ]:
def mediapipe_detection(img,model) : 
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img.flags.writeable = 0
    results = model.process(img)
    img.flags.writeable = 1
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    return img, results

In [ ]:
def draw_landmarks(img,results):
    mp_drawing.draw_landmarks(img,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(img,results):
    # Draw face connections 
    mp_drawing.draw_landmarks(img,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              # dots color
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              # connection lines color
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                              
                             )
    # Draw Body land marks 
    mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                              
                             )
    # Draw left hand landmarks
    mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                              )
    # Draw right hand landmarks
    mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                              )

In [ ]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=.5,min_tracking_confidence=.5) as holistic:
#     while cap.isOpened():
#         ret,frame = cap.read()
        
#         # make detecitons 
#         img, results = mediapipe_detection(frame,holistic)
#         print(results)
        
#         # drawing land marks
#         draw_styled_landmarks(img,results)
         
#         cv2.imshow('Video',img)
#         if cv2.waitKey(1) & 0xFF ==ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

# Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z,] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)  
    rh = np.array([[res.x,res.y,res.z,] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)  
    return np.concatenate([pose,face,lh,rh])

# Setup Folders for collection

In [ ]:
#  Path fo exportinh data , numpy arrays
DATA_PATH = os.path.join('MP_DATA')

#  actions that we try to detect
actions  = np.array(['HELLO',"THANKS","VICTORY"])

# 30 videos worth of data
no_sequences= 30

#  videos are going to be 30 frames in length
sequence_lenght = 30 

In [ ]:
 for action in actions:
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(DATA_PATH,action, str(sequence)))
            except:
                pass

# Collect keypoint values for training and testing

In [ ]:
# cap = cv2.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=.5,min_tracking_confidence=.5) as holistic:
# #     Loop through actions
#     for action in actions:
# #         loop through sequences aka videos
#         for sequence in range(no_sequences):
# #         Loop through video lrnght aka sequence length
#             for frame_num in range(sequence_lenght):
            
#                 ret,frame = cap.read()

#                 # make detecitons 
#                 img, results = mediapipe_detection(frame,holistic)
#                 print(results)

#                 # drawing land marks
#                 draw_styled_landmarks(img,results)
                
# #                  wait logic 
#                 if frame_num==0:
#                     cv2.putText(img,'STARTING COLLECTING',(120,200),
#                                 cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),1,cv2.LINE_AA)
#                     cv2.putText(img,'Collecting Frames for {} Video Number {}'.format(action,sequence),(15,12),
#                                 cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
#                     cv2.waitKey(2000)
#                 else:
#                     cv2.putText(img,'Collecting Frames for {} Video Number {}'.format(action,sequence),(15,12),
#                                 cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)                    
# #                 New export keypoints 
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
#                 np.save(npy_path,keypoints)
            
            
#                 cv2.imshow('Video',img)
#                 if cv2.waitKey(1) & 0xFF ==ord('q'):
#                     break
#     cap.release()
#     cv2.destroyAllWindows()

In [ ]:
# cap.release()
# cv2.destroyAllWindows()

# Preprocess Data and Create Labels and features

In [ ]:
%pip install scikit-learn

In [ ]:
!pip install tensorflow

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num ,label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequencs, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_lenght):
            res= np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequencs.append(window)
        labels.append(label_map[action])

In [ ]:
sequencs

In [ ]:
X = np.array(sequencs)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
labels

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.05)

# Build and train LSTM Neural Networks

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation = "relu",input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation = "relu"))
model.add(LSTM(64,return_sequences=False,activation = "relu"))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [ ]:
res = [.7,.2,.1]

In [ ]:
actions[np.argmax(res)]

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=2000,callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = np.argmax(model.predict(X_test))

res[0

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
y_hat_labels = [np.argmax(i) for i in y_hat]

In [ ]:
actions[np.argmax([y_test[1]])]

# Save Weights

In [ ]:
model.save('Actions.keras')

In [ ]:
model.load_weights('Actions.keras')

In [ ]:
import pickle
with open('Banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

# Evaluation using confusion Matrix and accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [ ]:
yhat = model.predict(X_train)
ytrue = np.argmax(y_train,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
cm = multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
cm

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm[0],annot=True,fmt='d')
plt.title("HELLO")
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm[1],annot=True,fmt='d')
plt.title("THANK YOU")
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm[2],annot=True,fmt='d')
plt.title("VICTORY")
plt.xlabel('Predicted')
plt.ylabel('True')

# test in real time

In [ ]:
#  New detection variables
sequence = []
sentence = []
threshold=.4
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=.5,min_tracking_confidence=.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # make detecitons 
        img, results = mediapipe_detection(frame,holistic)
#         print(results)
        
#         ########################### prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence  = sequence[:30]
        
        if len(sequence)==30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])

            
#          ###############   Viz logic ##################
        if res[np.argmax(res)] > threshold:
            if len(sentence) >0 :
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
        if len(sentence)>5:
            sentence  =sentence[-5:]
        
        cv2.rectangle(img,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(img,' '.join(sentence),(3,30),
                   cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv2.LINE_AA)


        # #################################### drawing land marks
        draw_styled_landmarks(img,results)
         
        cv2.imshow('Video',img)
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
np.expand_dims(X_test[0],axis=0).shape

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res,actions,img,colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        img, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
#         draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # #### Viz probabilities
            img = prob_viz(res, actions, img, colors)
            
        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(img, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', img)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()